In [23]:
from IPython.display import display, clear_output
import time
import random
import ipywidgets as widgets
from jupyter_ui_poll import ui_events
import requests
from bs4 import BeautifulSoup
import threading
import json

# This function sends the data collected in a dictionary into a Google form via url
def send_to_google_form(data_dict, form_url):
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

global_output = widgets.Output()
display(global_output)

def clear_output_wrapper():
    with global_output:
        clear_output()

# User info will be stored in this dictionary
user_info = {}

# This function runs before the test
def ask_for_info(): 
    
    with global_output:
        instructions = widgets.HTML(value = """ 
            <div style='font-size: 16px;'><b>Please input your credentials below</b></div>
            <br><i>Guidance</i><br>
            To generate an anonymous 4-letter unique user identifier please enter:<br>
            - 2 letters based on the initials (first and last name) of a childhood friend<br>
            - 2 letters based on the initials (first and last name) of a favourite actor / actress<br>
            <i>e.g., if your friend was called Peter Pan and film star was Brad Pitt<br>
            then your unique identifier would be PPBP.</i><br><br>""")

        # Asks the user to put in their anonymous ID, age, and select their sex
        user_id = widgets.Text(description = "ID:")
        user_age = widgets.IntText(description = "Age:") 
        sex_description = widgets.HTML(value = """<div style = 'font-size: 14px;'>Sex</div>""")
        user_sex = widgets.ToggleButtons(options=['F', 'M'])
        
        # This is the data submission button
        submit = widgets.Button(description = "Submit", layout = widgets.Layout(align_items='center', width='auto', margin = '10px')) 
        submit.style.button_color = '#5571AC'

        # This function runs after user clicks the submission button
        # It stores user's input in the widgets
        # clears the interface from the screen, and continues to the test
        def info_storage(btn):
            user_info['user_id'] =  user_id.value
            user_info['user_age'] = user_age.value
            user_info['user_sex'] = user_sex.value
            
            clear_output_wrapper()
            time.sleep(0.5)
            pre_test_info()
        
        submit.on_click(info_storage)

        credentials_vbox = widgets.VBox([instructions, user_id, user_age, sex_description, user_sex, submit], layout=widgets.Layout(align_items='center')) # Create a vertical box to hold the widgets

        # Display the widgets
        display(credentials_vbox)

def pre_test_info():

    with global_output:
        # This part tells the user what to expect in the test to increase the representativeness of the data collected
        start_guide = widgets.HTML(value = """
            <b>This is an approximate number system (ANS) test. </b><br><br>
            / Instructions /<br>
            For each trial, a test image will be shown for <b>0.75 seconds</b>. <br>
            2 buttons will appear on the top, displaying "Left" and "Right". <br>
            You need to choose the side with <b>more dots</b> <br>
            and click on the corresponding button <b>as fast as you can</b>. <br><br>
            <span style = "color: #5571AC;"><b>Click the start button when you are ready to go.</b></span>""")
        start = widgets.Button(description = "Start", layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', margin='10px'))
        start.style.button_color = '#5571AC'
        start_vbox = widgets.VBox([start_guide, start], layout = widgets.Layout(align_items='center'))
        display(start_vbox)
    
        def start_test(btn):
            clear_output_wrapper()
            time.sleep(0.5)
            test_start_time = time.time()
            run_test(test_start_time)
        
        # The test starts when user clicks the start button
        start.on_click(start_test)

def ask_for_consent(responses_dict):

    with global_output:
        # Asks the user for consent to upload the test results after the test
        data_consent = widgets.HTML(value = """<b>DATA CONSENT INFORMATION:</b><br>
        Please read:<br>
        we wish to record your response data<br>
        to an anonymised public data repository.<br>
        Your data will be used for educational teaching purposes<br>
        practising data analysis and visualisation.<br>
        <span style = 'color: #5571AC;'><b>Would you give us consent to record your test results?</b><br></span>""")
    
        choice = widgets.ToggleButtons(options = ['Yes', 'No'], layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', width='auto', margin = '10px'))
    
        # This is the consent submission button
        submit = widgets.Button(description = "Submit", layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', width='auto', margin = '10px'))
        submit.style.button_color = '#5571AC'
    
        # This variable will store user consent
        consent = []
        
        # This function runs after user clicks the submission button
        # It displays the test results to the user
        # and uploads all results to google form if user gives consent
        def judge_consent(btn):
            consent.append(choice.value)
            clear_output_wrapper()
            time.sleep(0.5)
    
            # This contains the test results that users might be interested in
            result = widgets.HTML(value = f"""Thanks for taking the test, user <span style = 'color: #5571AC;'>{responses_dict['user_id']}</span> :D<br>
            You took <b>{round(responses_dict["Time_Taken"]/60,2)} minutes</b> to complete the test.<br>
            The average response time is <b>{round(responses_dict["Average_response_time"], 2)} seconds</b>.<br>
            Your score is <b>{responses_dict["Total_Correct"]} out of 64</b>.<br>""", layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', width='500px'))
            
            if "Yes" in consent:
                
                # Send the data collected in the outcome to the Google form upon consent
                form_url = "https://docs.google.com/forms/d/e/1FAIpQLSfybwZCx7dJd8O9T_gNb3Am11ghlJK77FL3OEGrX93vE9JQ2Q/viewform?usp=sf_link"
                send_to_google_form(outcome, form_url)
                
                text = widgets.HTML(value = """<br>Thanks for your participation.<br>
                Your data has been uploaded.<br>
                Please contact philip.lewis@ucl.ac.uk<br>
                if you have any questions or concerns<br>
                regarding the stored results.""")
    
                # This displays the test results along with a confirmation of data upload
                result_vbox = widgets.VBox([result, text])
                display(result_vbox)
                
            elif "No" in consent:
                # Data not uploaded if no consent is given
                
                text = widgets.HTML(value = "<br>No problem, we hope you enjoyed the test!", layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', width='500px'))
                
                # This displays the test results along with a confirmation of data not upload
                result_vbox = widgets.VBox([result, text])
                display(result_vbox)
        
        submit.on_click(judge_consent)
        
        consent_vbox = widgets.VBox([data_consent, choice, submit], layout=widgets.Layout(align_items='center'))
    
        # Display the widgets
        display(consent_vbox)

# This is the main function for running the test
def run_test(test_start_time):
    with global_output: 
        
        # Creates a dictionary to store the number of correct and wrong responses for each Weber fraction (w)
        Judge = {"0.111C":0, "0.125C":0, "0.167C":0, "0.333C":0,
                 "0.111W":0, "0.125W":0, "0.167W":0, "0.333W":0}

        # Images were first sorted in ratio for more dots on the right and left respectively
        # The first 32 images have more dots on the right, the next 32 images have more dots on the left
        # The images were then sorted for the ratio of dots on the 2 sides
        # Afterwards the images were sorted for the number of dots on the 2 sides
        
        # Create lists to store the list of images with each number of dots
        l1 = []
        l2 = []
        l3 = []
        l4 = []
        l5 = []
        l6 = []
        l7 = []
        l8 = []
        list_of_lists = [l1, l2, l3, l4, l5, l6, l7, l8]
    
        # Define a function to store images
        def image_dots_num_store(list):
            for i in range(1, 5):
                i = i+4*list_of_lists.index(list)
                list.append(i)
                i += 32
                list.append(i)
            
            return list

        # Call the function on each dots number list
        for i in list_of_lists:
            image_dots_num_store(i)

        # Concatenate lists to obtain the lists for each Weber fraction (w)
        l111 = l1 + l4
        l125 = l3
        l167 = l2 + l6
        l333 = l5 + l7 + l8

        # Function to find the Weber fraction (w) for each image
        def find_w_key(value):
            w_dict = {"0.111": l111,
                      "0.125": l125,
                      "0.167": l167,
                      "0.333": l333
                      }

            # For each Weber fraction, if the image index is stored in the value,
            # return the Weber fraction
            for w in w_dict:
               if value in w_dict[w]:
                  return w
            
        # Create a list of image paths and image indexes
        image_list = []
        for i in range(1, 4):
            image_list.append([f'ANS_Pics/ANS ({i}).JPG', i])

        # Copy the image list and shuffle the copy randomly
        random_image_list = image_list.copy()
        random.shuffle(random_image_list)

        # Button widgets setup
        image = widgets.Image(layout=widgets.Layout(align_items='center', width='500px')) # Create image widget
        left = widgets.Button(description="Left", layout = widgets.Layout(flex_flow='column', width='250px', height='50px', align_items='center', margin='5px')) # Create left button widget
        right = widgets.Button(description="Right", layout = widgets.Layout(flex_flow='column', width='250px', height='50px', align_items='center', margin='5px')) # Create right button widget
        buttons_panel = widgets.HBox([left, right], layout = widgets.Layout(align_items='center'))

        # Create event to signal button click
        button_clicked = threading.Event()

        # time_up = False
        timer = None
        
        def response_timer(max_time):
            start_time = time.time()
            time_up = False
            
            while not time_up and (time.time() - start_time) < max_time:
                time.sleep(0.1)  # Short sleep to prevent high CPU usage
            if not time_up:
                time_up = True

            buttons_panel.close()  # Hide the buttons
            return time_up

        def register_choice(btn):
            time_up = True

            user_choice.append[btn.description]
            
            return time_up, user_choice

        # Register button click event handlers
        left.on_click(register_choice)
        right.on_click(register_choice) 

        no_click = 0 # Create a variable that stores the number of times the user didn't click in time
        total_response_time = 0 # Create a variable to measure the cumulative time taken when user clicks in time
        click = 0 # Create a variable to measure the number of times user clicks in time
        user_choice = [] # Create a list to store user's choice
        
        for i in image_list:
            # Reset user choice and buttons for each image
            time_up = False
            user_choice = []
            image_path = i[0]
            image_index = i[1]

            # Display image
            with open(image_path, 'rb') as f:
                image.value = f.read()
            display(image)

            # Hide image after displaying for 0.75s
            time.sleep(0.75)
            image.close()

            # Show choice buttons and start timing for user response
            display(buttons_panel)
            start_time = time.time()

            # Wait for user response until 3 seconds
            timer = threading.Thread(target = response_timer, args = (3))
            timer.start()

            if user_choice != []:
                
                # If user has made a choice in time, add response time to total
                total_response_time += time.time() - start_time
    
                # Add to total number of clicked in time
                click += 1
    
                # Call the predefined function to obtain image info
                info = find_w_key(image_index)
    
                # If user choice is correct, add to correct count
                if 'Right' in user_choice and image_index <= 32:
                    Judge[f'{info}C'] += 1
                                
                elif 'Left' in user_choice and image_index > 32:
                    Judge[f'{info}C'] += 1
                    
                # If user is wrong, add to wrong count
                else:
                    Judge[f'{info}W'] += 1

                click += 1

            else:
                # If user has not made a choice, add to no click count
                no_click += 1
    
                buttons_panel.close()
                total_response_time += time.time() - start_time
            
            # Intertrial interval
            time.sleep(1.5)

        # Calculate the total correct and wrong responses
        # Calculate the rate of correct responses
        # Calculate the total time taken for the test
        # Calculate the average response time
        total_correct = 0
        total_wrong = 0
        
        for key in Judge:
            if "C" in key:
                total_correct += Judge[key]
            if "W" in key:
                total_correct += Judge[key]
        
        if total_correct + total_wrong > 0:
            correct_rate = total_correct / (total_correct + total_wrong)
        else:
            correct_rate = 0

        total_time = time.time() - test_start_time
        
        if click > 0:
            average_response_time = total_response_time / click
        else:
            average_response_time = "Foul"

        # Create a dictionary of all responses and user info
        responses_dict = {
            "Not_clicked_in_time": no_click,
            "Total_Correct": total_correct,
            "Total_Wrong": total_wrong,
            "Correct_Rate": correct_rate,
            "Time_Taken": total_time,
            "Total_response_time": total_response_time,
            "Average_response_time": average_response_time
            }

        for i in Judge:
            responses_dict[i] = Judge[i]

        for i in user_info:
            responses_dict[i] = user_info[i]

        ask_for_consent(responses_dict)

# This runs the entire test
ask_for_info()

Output()